In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [ ]:
# leitura dos dados
elec_grid = pd.read_csv("Data_for_UCI_named.csv")
indices = elec_grid.index.values
colunas = list(elec_grid.columns)

In [ ]:
# distribuição das amostras
elec_grid['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [ ]:
elec_grid['stabf'] = stabf

In [ ]:
def selecao_treino_teste(dataset, indices):
  """
  Divide a base em treino e teste

  Params:
    - dataset: dataframe do pandas
    - indices: indices da base de dados original
  return:
    - df_treino
    - df_teste
    - y_treino
    - y_teste
  """
  # escolhe os indices de treino aleatoriamente
  idx_treino = np.random.choice(indices,size=8000, replace=False)
  # obtem os indices de teste
  idx_teste = list(set(indices).difference(idx_treino))
  
  # selecao dos dataframes de treino e teste
  df_treino = elec_grid.iloc[idx_treino]
  df_teste = elec_grid.iloc[idx_teste]
  y_treino = df_treino['stabf'].values
  y_teste = df_teste['stabf'].values
  
  return df_treino, df_teste, y_treino, y_teste

# a)

In [ ]:
def acuracia(y_real, y_pred):
  """
  Calcula a acuracia com base na predicao de um modelo
  """
  comparacoes = y_real==y_pred
  acc = (sum(comparacoes)/len(comparacoes))*100
  return acc

def revocacao(y_real, y_pred):
  """
  Calcula a revocação com base na predição de um modelo
  """

  TP = 0
  FN = 0
  for yr, yp in zip(y_real, y_pred):
      if (yr=='stable'):
          if (yp=='stable'): 
              TP+=1
          else:
              FN+=1
  R = (TP/(TP+FN))*100
          
  return R  

def precisao(y_real, y_pred):
  """
  Calcula a precisão com base na predição de um modelo
  """
  TP = 0
  FP = 0
  for yr, yp in zip(y_real, y_pred):
      if (yr=='stable'):
          if (yp=='stable'): 
              TP+=1
      else:
          if(yp=='stable'):
              FP+=1
  P = (TP/(TP+FP))*100
  return P          

def treino_rocchio(X_treino, y_treino):
  """
  Realiza o treino do Rocchio

  Params:
    - X_treino: base de treino
    - y_treino: classes da base de treino
  """  

  colunas = list(X_treino.columns)
  # classes presentes na base se dados
  y_classes = np.unique(y_treino)
  u_is = []
  # calcula a média dos atributos de cada classe
  for y in y_classes:
      subsetX = X_treino[X_treino['stabf']==y]
      u_i = subsetX[colunas[:-1]].mean().values
      u_is.append(u_i)
  
  return y_classes, u_is

def classificacao_rocchio(X_teste, y_classes, u_is, inv_cov):
  """
  Realiza a classificação com um modelo Rocchio

  Params:
    - X_teste: base de teste
    - y_classes: valores únicos das classes disponíveis na base de treino
    - u_is: medias dos atributos
    - inv_cov: inversa da matriz de covarâncias dos dados
  
  return:
    - pred_classes: classificação da base de teste
  """

  pred_classes = []
  # classifica cada amostra presente em X
  for x in X_teste:
      distancias = [mahalanobis(x, u, inv_cov) for u in u_is]
      idx = np.argmin(distancias)
      pred_classes.append(y_classes[idx])
  return pred_classes

def mahalanobis(a_x,b_x, inv):
  """
  Distância de mahalanobis
  """
  x_mu = a_x-b_x
  left = np.dot(x_mu, inv)
  mahal = np.dot(left, x_mu.T)
  return np.sqrt(mahal)

In [ ]:
# pega os valores da base de dados
X = elec_grid[colunas[:-1]].values
start_time = time.monotonic()
# calcula a matriz de covariancia e sua inversa
cov = np.cov(X.T)
inv_cov = np.linalg.inv(cov)

metricas = {
    "acuracia":[],
    "revocacao":[],
    "precisao": []
}
# execução dos 5 testes
for i in range(5):
  # aleatorioamente seleciona as base de treino e teste
  df_treino, df_teste, y_treino, y_teste = selecao_treino_teste(elec_grid, indices)
  # realiza o treino do rocchio
  y_classes, u_is = treino_rocchio(df_treino, y_treino)
  # base de teste
  X_teste = df_teste[colunas[:-1]].values
  # classifica a base de teste
  predy_teste = classificacao_rocchio(X_teste, y_classes, u_is, inv_cov)
  
  ## calcula as metricas
  acc = acuracia(y_teste,predy_teste)
  rev = revocacao(y_teste, predy_teste)
  pre = precisao(y_teste, predy_teste)
  metricas['acuracia'].append(acc)
  metricas['revocacao'].append(rev)    
  metricas['precisao'].append(pre)
acc = np.mean(metricas["acuracia"])
pre = np.mean(metricas["precisao"])
rev = np.mean(metricas["revocacao"])

end_time = time.monotonic()

print ("Acurácia | Precisão\t| Revocação")

print ("{:.2f}\t|\t{:.2f}\t|\t{:.2f}".format(acc,pre,rev))
print ("Tempo: {}".format(end_time - start_time))

Acurácia | Precisão	| Revocação
93.03	|	85.32	|	98.05
Tempo: 0.4464843760000008


In [ ]:
minutos = 0.40597203099997614/60
print ("{:.3f}".format(minutos))

0.007


In [ ]:
df_treino, df_teste, y_treino, y_teste = selecao_treino_teste(elec_grid, indices)
y_classes, u_is = treino_rocchio(df_treino, y_treino)
X_teste = df_teste[colunas[:-1]].values

# b)

In [ ]:
def dist_euclidiana(x, y):
  """
  Calcula a distância euclidiana
  """
  return np.linalg.norm(x-y)

def get_indices(a,k):
  idx = []
  for i in range(k):
    indice = np.argmin(a)
    idx.append(indice)
    a[indice] = np.inf
  return idx   
    
def get_class(y):
    pred = max(set(y), key=list(y).count)
    return pred

In [ ]:
df_treino, df_teste, y_treino, y_teste = selecao_treino_teste(elec_grid, indices)
df_treino_p1 = df_treino.iloc[:4000]
df_treino_p2 = df_treino.iloc[4000:]
X_treino_p1 = df_treino_p1[colunas[:-1]].values
X_treino_p2 = df_treino_p2[colunas[:-1]].values
y_treino_p1 = df_treino_p1['stabf'].values
y_treino_p2 = df_treino_p2['stabf'].values

In [ ]:
def calc_dist_matriz(X,Y):
    """
    Calcula a matriz de distancia entre os valores de X 
    para cada valor em Y
    """
    m = X.shape[0]
    n = Y.shape[0]
    M = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            M[i][j] = dist_euclidiana(X[i], Y[j])
    return M       

def KNN(matriz_distancias, y_treino, y_teste, k):  
    vizinhos = [np.argsort(matriz_distancias[i])[:k] for i in range(len(matriz_distancias))]
    y_pred = [get_class(y_treino[v]) for v in vizinhos]
    return y_pred

def melhor_K(elec_grid, indices):
  """
  Obtem o melhor valor de k para o KNN usando a métrica acurácia

  """
  dict_metricas = {}

  for i in range(5):
    print ("Execução {}".format(i+1))
    # otbem novas bases de treino e teste
    df_treino, df_teste, y_treino, y_teste = selecao_treino_teste(elec_grid, indices)
    # divide a base de treino em duas bases iguais
    df_train = df_treino.iloc[:4000]
    df_vali  = df_treino.iloc[4000:]
    
    X_train = df_train[colunas[:-1]].values
    X_vali  = df_vali[colunas[:-1]].values
    
    y_train = df_train['stabf'].values
    y_vali  = df_vali['stabf'].values

    # calcula matriz de distância
    matriz_dist = calc_dist_matriz(X_vali, X_train)
    for k in range(1,12,2):
      dict_metricas.setdefault(k,{"a":[], "p":[], "r":[]})
      # obtem a classe dos k vizinhos mais próximos
      y_pred = KNN(matriz_dist, y_train, y_vali, k)

      # calcula e salva as métricas
      acc = acuracia(y_vali, y_pred)
      pre = precisao(y_vali, y_pred)
      rec = revocacao(y_vali, y_pred)
      dict_metricas[k]["a"].append(acc)
      dict_metricas[k]["p"].append(pre)
      dict_metricas[k]["r"].append(rec)
  melhoracc = 0
  melhork = 1
  print ("K\tAcc\t|\tPrec\t|\tRecall")
  for k in dict_metricas:
    ac = np.mean(dict_metricas[k]["a"])
    pr = np.mean(dict_metricas[k]["p"])
    re = np.mean(dict_metricas[k]["r"])
    if (ac > melhoracc):
      melhork=k
      melhoracc = ac
  print ("{} \t{:.2f}\t|\t{:.2f}\t|\t{:.2f}".format(melhork, ac, pr, re))
  return melhork

In [ ]:
start_time = time.monotonic()
k = melhor_K(elec_grid, indices)
end_time = time.monotonic()
print ("Tempo: {}s".format(end_time - start_time))

Execução 1
Execução 2
Execução 3
Execução 4
Execução 5
K	Acc	|	Prec	|	Recall
11 	78.80	|	75.41	|	61.89
Tempo: 784.4886422199999s


In [ ]:
minutos = 784.4886422199999/60
print ("{:.3f}".format(minutos))

13.075


# c)

In [ ]:
def edit_kNN(X,Y, k=1):
  """
  Edit kNN com inserção sequencial

  Params:
    - X: base de treino
    - Y: classes da base de treino
    - k: valor de k  a ser aplicado
  return:
    - adicionados: indices dos valores de X a serem mantidos no KNN
  """
  D_ = []
  Y_ = []
  # calcula a matrizde distância
  md = calc_dist_matriz(X,X)
  # faz uma copia da matriz com todos os valores = inf
  md_ = np.ones_like(md)*np.inf
  indices = np.arange(len(X))
  # escolhe os indices aleatoriamente para o algoritmo
  indices = np.random.choice(indices,replace=False, size=len(X))
  adicionados = []
  # percorre os indices
  for i in indices:
    # primeiro indice sempre é adicionado
    if(len(adicionados)==0):
      md_[i] = md[i]
      md_[:,i] = md[i]
      adicionados.append(i)
    else:
      # evita que o proprio elemento seja visto com mais próximo
      md_[i][i] = np.inf
      # k vizinhos
      vizinhos = np.argsort(md_[i])[:k]
      # classes dos vizinhos
      y_pred = get_class(Y[vizinhos])
      # verifica se foi classificado corretamente
      if Y[i] != y_pred:
        md_[i] = md[i]
        md_[:,i] = md[i]
        adicionados.append(i)
  return adicionados

In [ ]:
start_time = time.monotonic()
k = 11
for i in range(5):
  print ("Execução {}".format(i))
  acuracias = []
  precisoes = []
  revocacoes = []
  # Seleção das bases de treino e teste
  df_treino, df_teste, y_train, y_teste = selecao_treino_teste(elec_grid, indices)
  # selação dos valores
  X_treino = df_treino[colunas[:-1]].values
  X_teste = df_teste[colunas[:-1]].values
  
  # seleção das amostras a serem adicionadas
  adicionados = edit_kNN(X_treino, y_train, k=k)
  dict_metricas = {}
  # seleciona as amostras de Xtreino
  X_menor = X_treino[adicionados]
  # seleção dos ytreino
  y_menor = y_train[adicionados]
  # calcula a matriz de distâncias usando a base reduzida
  matriz_dist = calc_dist_matriz(X_teste, X_menor)
  # classifica a base de teste 
  y_pred = KNN(matriz_dist, y_menor, y_teste, k)  
  # calcula métricas
  acc = acuracia(y_teste, y_pred)
  pre = precisao(y_teste, y_pred)
  rec = revocacao(y_teste, y_pred)
  acuracias.append(acc)
  precisoes.append(pre)
  revocacoes.append(rec)

end_time = time.monotonic()
print ("K\tAcc\t|\tPrec\t|\tRecall")
print ("{} \t{:.2f}\t|\t{:.2f}\t|\t{:.2f}".format(k, np.mean(acuracias), np.mean(precisoes), np.mean(revocacoes)))
print ("Tempo: {}".format((end_time - start_time)/60))

Execução 0
Execução 1
Execução 2
Execução 3
Execução 4
K	Acc	|	Prec	|	Recall
11 	73.15	|	63.09	|	65.40
Tempo: 73.23720541679998


In [ ]:
minutos = 3181.795633755/60
print ("{:3f}".format(minutos))

53.029927


# d)

**Tabela com a média dos resultados de 5 execuções dos modelos implementados, mostrando a Acurácia, Precisão, Revocação e Tempo de execução de cada modelo**.

|                   | Rocchio | kNN    | Edit kNN |
|-------------------|---------|--------|----------|
| Acurácia          | 93.27   | 78.80  | 73.15    |
| Precisão          | 85.52   | 75.41  | 63.09    |
| Revocação        | 98.36   | 61.89  | 65.40    |
| Tempo de Execução (min) | 0.007   | 13.075 | 53.02    |

O Rocchio e o Edit kNN foram treinados com bases com 8000 amostras de treino e 2000 de teste. O kNN foi treinado utilizando uma base 4000 amostras de treino e 4000 de teste.

A tabela mostra que o Rocchio foi o modelo com melhores avaliações para todas as métricas utilizadas, além de ser o modelo com execução mais rápida (0.007 minutos).

O kNN, apesar de mais robusto, teve desepenho inferior ao Rocchio, e teve Revocação inferior ao Edit kNN.

O Edit kNN teve execução mais demorada que o kNN, o que é explicado pelo fato de ter sido treinado com uma base de dados maior. Enquanto o KNN foi treinado com 4000 amostras de treino e teste, a seleção das amostras do Edit KNN utilizou 8000 amostras de treino.

Por fim, durante o cálculo das métricas, utilizou-se a classe "stable" como sendo a classe "positiva". Como essa é a classe minoritária, podemos afirmar que a métrica de acurácia utilizada é relevante. 